In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# GUI for Setting Variables in Notebooks

```{Note}
This GUI makes use of the Python package `ipywidgets`. Installing `ipywidgets` also installs a Jupyter
notebook extension. Therefore, you will have to restart your notebook server after installing `ipywidgets`.
```

This is an experimental interface to make it easy for users to set the values of variables in an OpenMDAO model
when using a Jupyter Notebook.

There are some limitations:

* no support for arrays
* for large models, creating the GUI can take a long time
* not tested with Jupyter Lab

There is a single function to create the GUI.

```{eval-rst}
.. autofunction:: openmdao.visualization.set_values_gui.set_values_gui
    :noindex:
```

Here is an example of how to use it with a simple model.


In [ ]:
from openmdao.visualization.set_values_gui.set_values_gui import set_values_gui
import openmdao.api as om

class SpeedComp(om.ExplicitComponent):
    """Simple speed computation from distance and time with unit conversations."""

    def setup(self):
        self.add_input('distance', val=1.0, units='km')
        self.add_input('time', val=1.0, units='h')
        self.add_output('speed', val=1.0, units='km/h')

    def compute(self, inputs, outputs):
        outputs['speed'] = inputs['distance'] / inputs['time']

prob = om.Problem()
prob.model.add_subsystem('c1', SpeedComp(), promotes=['*'])
prob.model.add_subsystem('c2', om.ExecComp('f=speed',speed={'units': 'm/s'}), promotes=['*'])

prob.model.set_input_defaults('distance', val=1., units='m')
prob.model.set_input_defaults('time', val=1., units='s')

prob.setup()
prob.final_setup()


In [ ]:
ui = set_values_gui(prob, vars_to_set=[], initial_depth = 1, ivc_only= False, debug=False)

To display it, just enter the GUI object as the last item in the cell.

In [ ]:
ui

The GUI should look like this initially.

![gui_before_vars_displayed](images/gui_before_vars_displayed.png)

The left part of the GUI displays the model hierarchy.

You can use the + and - widgets to collapse and expand
the model tree.

There are 3 kinds of icons in this display. The envelope icon is for OpenMDAO groups. The plug icon
represents OpenMDAO Components. Finally the eyeball icon represents variables. If a slash is through this icon, the
variable does not have a widget to set its value in the middle section, Model Variables.
Click on that icon to create a widget for setting its value.

The variables are shown relative their promoted level in the model hierarchy.

The middle part of the GUI shows the widgets for entering values of
variables and their units. The values for the variables and units are
set when the Enter key is struck while in the data entry widget
or the focus is taken away from that data entry widget. The combination of the
value and units is set simultaneously when either data entry widget is modified.

The right part of the GUI has a single button, `Refresh`. The GUI is currently "one-way" in the sense that the GUI lets you
set the model variables but if the model variables are set outside of the GUI, the GUI no longer is in sync with
the model variables. Clicking on the Refresh button syncs the GUI display with all the model variables and units.

As an example, after requesting that widgets be displayed to set the values of `speed`, `distance`, and `time`,
the GUI should look like this.

![diagram of a simple circuit with two resistors and one diode](images/gui_after_vars_displayed.png)

The user can then set the values and units for those 3 variables.